## Script to calculate the long term and shorter term runoff ratio from the VIC 

In [1]:
import numpy as np
import pandas as pd
from IPython import parallel as pr

In [2]:
c = pr.Client() # start the parallel environment

In [3]:
c.ids # check the number of engines available

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [4]:
view = c.load_balanced_view() # access the first 4 engines

In [5]:
%%px

# load the working environment

import pandas as pd
import numpy as np
dates = pd.read_pickle('/Volumes/data/Theo/projects/Budyko_vic/timecode.pcl')
fluxes_columns = ['y','m','d','ET','R','BF','sm1','sm2','sm3','SWE','Cs','Qs','Ql','Qg','NR','PEText','PETtrc','PETsrc']
forcing_columns = ['P','Tmax','Tmin','W']
# bring in the data frame of file paths
files = pd.read_pickle('/Volumes/data/Theo/projects/Budyko_vic/forcing_fluxes_filenames_lat_lon_index.df') 

cells = pd.DataFrame()
data = np.load('/Volumes/data/Theo/projects/Budyko_vic/data/VIC_cells_overlap.npz') # load the soil data

# loop through the data frame
for key in data.keys():
    cells[key] = data[key]

# temporal bounds of the Newman et al. [2015] data set
strt = '1980-1-1'
nd = '2011-12-23'    
    
# define a function to read in the flux file and return the sum of the discharge
def fluxproc(fl):
    flux = pd.read_table(fl, sep='\t', names = fluxes_columns) # read fluxes
    flux.index = pd.DatetimeIndex(dates)
    
    flux['discharge'] = flux.BF+flux.R # compute the total discharge [mm]
    
    return flux.loc[strt:nd,'discharge'].sum()

def forceproc(fr):
    force = pd.read_table(fr,sep=' ', names = forcing_columns) # read forcings
    force.index = pd.DatetimeIndex(dates)
    
    return force.loc[strt:nd,'P'].sum()

In [6]:
strt = '1980-1-1'
nd = '2011-12-23'    
    
# define a function to read in the flux file and return the sum of the discharge
def fluxproc(fl):
    flux = pd.read_table(fl, sep='\t', names = fluxes_columns) # read fluxes
    flux.index = pd.DatetimeIndex(dates)
    
    flux['discharge'] = flux.BF+flux.R # compute the total discharge [mm]
    
    return flux.loc[strt:nd,'discharge'].sum()

def forceproc(fr):
    force = pd.read_table(fr,sep=' ', names = forcing_columns) # read forcings
    force.index = pd.DatetimeIndex(dates)
    
    return force.loc[strt:nd,'P'].sum()

In [7]:
dates = pd.read_pickle('/Volumes/data/Theo/projects/Budyko_vic/timecode.pcl')
fluxes_columns = ['y','m','d','ET','R','BF','sm1','sm2','sm3','SWE','Cs','Qs','Ql','Qg','NR','PEText','PETtrc','PETsrc']
forcing_columns = ['P','Tmax','Tmin','W']

In [8]:
files = pd.read_pickle('./forcing_fluxes_filenames_lat_lon_index.df') # bring in the data frame of file paths

In [9]:
cells = pd.DataFrame()
data = np.load('./data/VIC_cells_overlap.npz') # load the vic cells and basins

# loop through the SWE data

for key in data.keys():
    cells[key] = data[key]

In [10]:
cells.head()

,basins,cells,percents
0,1013500,220561,0.03
1,1013500,221439,0.04
2,1013500,221442,0.07
3,1013500,221443,0.67
4,1013500,221444,0.74


In [11]:
basins = np.unique(cells.basins) # create a list of unique basins

In [12]:
len(basins)

677

In [13]:
# define a long term P function
def ltQ(basin):


    viccells = cells.loc[cells.basins==basin,'cells'] # pull the cell index numbers
    vicpercs = cells.loc[cells.basins==basin,'percents'] # pull the cell percents
    
    # create lists of file paths for forcing and flux files
    fluxfiles = []
    
    for cell in viccells:
        fluxfiles.extend(list(files.loc[files.indexer==cell,'flux'].as_matrix()))
    
    # create a data frame for each basin
    temp = pd.DataFrame({'cellid':viccells,'cellperc':vicpercs,'flux':fluxfiles}) 
    
    #temp['P'] = temp.force.map(forceproc)
    temp['Q'] = temp.flux.map(fluxproc)
    
    # assign column the weights that would be used for normal averaging
    temp['aveweight'] = 1./len(temp)
    
    # compute the final weight for each cell based on the number of cells and the
    # percent of the cell that falls within the basin
    temp['weight'] = temp.aveweight*temp.cellperc  
    
    temp['Q2'] = temp.Q*temp.weight # multiply Q by the final weight
    
    Q = temp.Q2.sum() # compute the sum of the weighted precipitation
    
    return basin,Q

In [14]:
res = view.map(ltQ,basins) # map the function to all the basins over the engines

In [15]:
res.ready()

False

In [47]:
res.progress

677

In [48]:
res.elapsed/60.

43.461681516666665

In [49]:
basins,LTQ = zip(*res.result)

In [51]:
np.savez_compressed('/Volumes/data/Theo/projects/Budyko_vic/data/ground_truth_ltq.npz',LTQ=LTQ,basin=basins)